# Corretor ortográfico
A ideia desse notebook é implementar corretor ortográfico automático pro texto das leis, a fim de analisar se isto impacta na qualidade da representação dos documentos

In [39]:
import pandas as pd
import matplotlib.pyplot as plt

In [40]:
# Importa arquivos
# Descomente pra usar no Google Colab
# from google.colab import files
# import os.path

arq_leis = 'leis.json'

# if (not os.path.isfile(arq_leis)):
#     uploaded = files.upload()

In [41]:
leis = pd.read_json(arq_leis)
leis.drop(['documento'], inplace=True, axis=1)
print(leis.info())
print(leis.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   titulo     6033 non-null   object
 1   categoria  6033 non-null   object
 2   resumo     6033 non-null   object
 3   texto      6033 non-null   object
dtypes: object(4)
memory usage: 188.7+ KB
None
titulo       6033
categoria       8
resumo       4961
texto        6029
dtype: int64


In [42]:
leis

,titulo,categoria,resumo,texto
0,"DECRETO Nº 8854, de 28 de fevereiro de 2013.",Decretos,DELEGA COMPETÊNCIA À SECRETARIA MUNICIPAL DE P...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
1,"DECRETO Nº 8849, de 25 de fevereiro de 2013.",Decretos,ABRE CRÉDITO SUPLEMENTAR AO ORÇAMENTO DO MUNIC...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
2,"DECRETO Nº 8853, de 27 de fevereiro de 2013.",Decretos,NOMEIA MEMBROS DO CONSELHO MUNICIPAL DE DESENV...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
3,"DECRETO Nº 8967, de 17 de julho de 2013",Decretos,ALTERA O QUADRO DE DETALHAMENTO DE DESPESA DO ...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
4,"DECRETO Nº 8982, de 30 de julho de 2013",Decretos,AUTORIZA O FUNCIONAMENTO DE ESTABELECIMENTOS C...,"O PREFEITO MUNICIPAL DE FEIRA DE SANTANA, Esta..."
...,...,...,...,...
6028,RESOLUÇÃO Nº 125/1980,Resoluções,DISPÕE SOBRE A CONCESSÃO DE TÍTULO DE CIDADÃO ...,Faço saber que a Câmara Municipal aprovou e eu...
6029,RESOLUÇÃO Nº 403/2003,Resoluções,AUTORIZA A MESA DIRETIVA DO PODER LEGISLATIVO ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6030,RESOLUÇÃO Nº 492/2014,Resoluções,INSTITUI A SEGUNDA SEMANA DO MÊS DE AGOSTO EM ...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."
6031,RESOLUÇÃO Nº 382/2001,Resoluções,CRIA A MEDALHA VEREADOR DIVAL FIGUEIREDO MACHA...,"A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado..."


In [43]:
# Exemplo de texto de lei
leis.loc[len(leis)-1, 'texto']

'A CÂMARA MUNICIPAL DE FEIRA DE SANTANA, Estado da Bahia, na conformidade do artigo 70, Inciso V, da Lei Municipal nº37, de 05 de Abril de 1990 e, artigos 287, § 2º e, 420, do Regimento Interno, promulga a seguinte Resolução:\n\nArt. 1ºDê-se aos dispositivos abaixo mencionados, da Resolução nº393/2002 - Regimento Interno, as seguintes redações:\n\n"Art. 7º A Mesa Diretora da Câmara compor-se-á do Presidente, Primeiro e Segundo Secretários, com mandato de 02 ( dois ) anos, admitida a recondução para a eleição subsequente.\n\n§ 4º Se, hora regimental, não estiver presente o Presidente, abrirá os trabalhos o Vice-Presidente ou, na falta deste, o Primeiro ou Segundo Secretários, na sequência, ou ainda, caso estes não estejam presentes, o Vereador mais votado nas eleições municipais."\n\n"Art. 33 Compete, privativamente, ao Vice-Presidente:"\n\n"Art. 36 ...\n\nI - ...\n\ne) acompanhar e supervisionar a Ata da Sessão, proceder a sua leitura e assiná-la depois do Presidente e do Vice-Presiden

In [44]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import unicodedata

def limpa_texto(text, string=True) -> str or list:
    if (type(text) == float):
        return ''

    # Remove pontuacao, digitos e espacos
    text = ' '.join(re.findall(r'\b[a-zÀ-ú]+\b', text.lower()))
    
    # Remove acentos, cedilhas etc
    # nfkd_form = unicodedata.normalize('NFKD', text)
    # text = ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

    # Remove stopwords
    my_stopwords = stopwords.words('portuguese')
    my_words = ['r', 'iii', 'i', 'ii', 'desta', 'v', 'iv', 'deste',
    'qualquer', 'b', 'das', 'seguintes', 'm', 'c', 'g', 'n', 'w',
    'k', 'l', 's', 't', 'z', 'd', 'j', 'p', 'y', 'f', 'u', 'x', 'q', 'h',
    'iv', 'ix']
    my_stopwords = my_stopwords + my_words

    text = [word for word in text.split() if word not in my_stopwords]

    if string:
        return ' '.join(text)
    else:
        return text

[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Semelhança de documentos

In [45]:
leis['texto_limpo'] = leis['texto'].apply(limpa_texto)

In [46]:
# Constrói matriz de documentos
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
vectorizer = CountVectorizer()
representacao_contagem = vectorizer.fit_transform(leis['texto_limpo'])
representacao_contagem

<6033x30091 sparse matrix of type '<class 'numpy.int64'>'
	with 619936 stored elements in Compressed Sparse Row format>

In [47]:
vocabulario = vectorizer.vocabulary_
print(vocabulario)

: 20499, 'parâmentros': 21078, 'mundança': 19307, 'translação': 28297, 'sancionará': 25835, 'vetará': 29331, 'obsoluta': 20050, 'po': 21979, 'incluidos': 15150, 'sucederá': 27143, 'residirão': 25040, 'responsábilidade': 25126, 'outorgadas': 20658, 'decadências': 8066, 'desigual': 8875, 'aumentando': 2992, 'aumentou': 2999, 'confisco': 6524, 'adquirinte': 686, 'transparências': 28329, 'creditadas': 7546, 'aumente': 2995, 'bimentre': 3681, 'orgânicaserão': 20504, 'aparecidos': 1888, 'instituidas': 15798, 'elaborá': 10305, 'excedem': 11829, 'corespondentes': 7302, 'comprendidos': 6126, 'incorrerão': 15232, 'municípais': 19335, 'definação': 8230, 'tratabalho': 28373, 'desapropriados': 8622, 'ecológíco': 10085, 'concesssão': 6300, 'primado': 22739, 'psicoativos': 23396, 'empregadoras': 10615, 'superdotados': 27245, 'recenseamento': 24074, 'institução': 15816, 'bianualmente': 3635, 'demogrática': 8399, 'inclusíve': 15163, 'abrigação': 129, 'aprovaçõa': 2103, 'cimentado': 5311, 'acessivel': 2

In [48]:
dicionario = []
with open('.dicio/palavras.txt') as f:
    dicionario = [i for i in f.read().splitlines()]
print(dicionario)
print(len(dicionario))

o', 'xantopirita', 'xantoplastídio', 'xantoplasto', 'xântopo', 'xantópode', 'xantoproteico', 'xantoproteína', 'xantopse', 'xantopsia', 'xantopsidracia', 'xantopsidrácico', 'xantopsina', 'xantopterina', 'xantóptero', 'xantóptico', 'xantopucina', 'xantopurpurina', 'xantoquimo', 'xantorramnina', 'xantorreia', 'xantorreico', 'xantorrizo', 'xantortito', 'xantos', 'xantose', 'xantosia', 'xantoso', 'xantospermo', 'xantóspilo', 'xantossarcoma', 'xantossiderito', 'xantossoma', 'xantostromina', 'Xantoxiláceas', 'Xantoxíleas', 'xantóxilo', 'xantungue', 'xanturia', 'xantúria', 'xantusiídeo', 'xanxão', 'Xanxerê', 'xanxerense', 'xaorô', 'xapacura', 'xapanã', 'xaponã', 'Xapuri', 'xapuriense', 'xaputa', 'xaque', 'xaque-mate', 'xaque-xaque', 'xaquear', 'xáquema', 'xaquetar', 'xaquiaísmo', 'xáquima', 'Xaquira', 'xar', 'xar-pei', 'xara', 'xará', 'xara-branca', 'xara-preta', 'xarada', 'xarafim', 'xarafo', 'xarapa', 'xarapim', 'xarau', 'xarda', 'xardoso', 'xarel', 'xarelete', 'xarém', 'xareta', 'xaréu', 'x

In [49]:
dicionario = list(set(map(limpa_texto, dicionario)))
len(dicionario)

316974

In [55]:
dicionario = dicionario[1:]
dicionario

['invisível',
 'laganhento',
 'genecólogo',
 'acetiltartárico',
 'inamuine',
 'antropologístico',
 'proteioquinase',
 'galvanizar',
 'cabamba',
 'talidomidopático',
 'purgável',
 'cigarras peito branco',
 'ibero céltico',
 'colicoplégico',
 'erucato',
 'chissila',
 'lordes mores',
 'canafístulas lagoa',
 'pneumemia',
 'pedra lua',
 'escobar',
 'coibitivo',
 'canil',
 'conseguinte',
 'lolardismo',
 'abelhas alemãs',
 'medusiforme',
 'páter',
 'colissepse',
 'orográfico',
 'semiotecnia',
 'caveirense',
 'célulazinha',
 'dccxli',
 'oquedê',
 'íeis',
 'abnodação',
 'pitexavássana',
 'abilgárdia',
 'pródromo',
 'feculométrico',
 'islândico',
 'trinca ferros verdadeiros',
 'pitanga cachorro',
 'arapaçu',
 'pneumectomia',
 'galos pluma',
 'escacha',
 'corvineiro',
 'tetérrimo',
 'siliconar',
 'conguês',
 'correger',
 'dexloxiglumida',
 'gilvan',
 'saca',
 'cambará folha miúda',
 'fibrogênico',
 'balizeiro',
 'heteropoliácido',
 'gordífida',
 'rotogravar',
 'break',
 'castelhana',
 'roboão',
 

In [57]:
comeca_o = [i for i in dicionario if (i[0:2]== 'us')]
comeca_o

['usadiço',
 'usovita',
 'usança',
 'usuários chave',
 'ustilaginale',
 'ustível',
 'usâmbio',
 'usbequistão',
 'ustilagíneo',
 'usufrutuário',
 'usnônico',
 'usucapiendo',
 'usuano',
 'usina',
 'uscardeu',
 'ustório',
 'usurpativo',
 'usável',
 'usina protótipo',
 'ustos',
 'ustarasita',
 'usucapto',
 'usnato',
 'usuários produtores',
 'ustilagináceo',
 'ustilaginoso',
 'ussila',
 'ustilaginal',
 'ustão',
 'usufruto',
 'usípete',
 'usurpatório',
 'ustrássana',
 'usp',
 'usneol',
 'us',
 'usbequistanês',
 'usnético',
 'ustulação',
 'usos',
 'usbequita',
 'usmar',
 'usufrutuar',
 'ustrina',
 'ussa',
 'usurário',
 'usucapiente',
 'ustulável',
 'usurpação',
 'usinável',
 'usineiro',
 'usnidínico',
 'usufruível',
 'usufrutueiro',
 'usnínico',
 'usuários',
 'usinas protótipo',
 'ussinguita',
 'usuário',
 'usufructuário',
 'usurpar',
 'usb',
 'usneína',
 'uscufe',
 'usnólico',
 'usinense',
 'ussia',
 'uscanense',
 'usagre',
 'useiro',
 'usneal',
 'usucapião',
 'uscardeiazinha',
 'usuário pro

In [51]:
palavras_erradas = [palavra for palavra in vocabulario if palavra not in dicionario]
print(len(palavras_erradas))
palavras_erradas

19320


['uso',
 'atribuições',
 'considerando',
 'denominado',
 'oferta',
 'aludido',
 'dispõe',
 'antidrogas',
 'estabelece',
 'outra',
 'desenvolvimento',
 'políticas',
 'drogas',
 'ilícitas',
 'decreta',
 'comad',
 'respectivas',
 'entrará',
 'revogadas',
 'disposições',
 'lucena',
 'legais',
 'contida',
 'detalhamento',
 'class',
 'programática',
 'econômica',
 'formatada',
 'técnica',
 'leismunicipais',
 'disponíveis',
 'despesas',
 'decorrentes',
 'presente',
 'correrão',
 'anulações',
 'dotações',
 'detalhadas',
 'contabilidade',
 'autorizada',
 'contábeis',
 'necessários',
 'cumprimento',
 'estratégica',
 'fomento',
 'política',
 'indicações',
 'formuladas',
 'entes',
 'governamentais',
 'integram',
 'nomeadas',
 'integrarem',
 'representando',
 'settdec',
 'márcia',
 'cristina',
 'campodônio',
 'eloy',
 'planejamento',
 'brito',
 'fraga',
 'maia',
 'luis',
 'venas',
 'universidade',
 'uefs',
 'celismara',
 'morais',
 'subaé',
 'cedraz',
 'vaccarezza',
 'baiana',
 'ebda',
 'caldas',
 

In [65]:
dic_ugc = []
with open('.dicio/dicio_UGC.txt') as f:
    dic_ugc = [linha.split(',')[0] for linha in f]
len(dic_ugc)

549053

In [66]:
palavras_erradas_ugc = [palavra for palavra in palavras_erradas if palavra not in dic_ugc]
print(len(palavras_erradas_ugc))
palavras_erradas_ugc

5171


['comad',
 'class',
 'leismunicipais',
 'settdec',
 'campodônio',
 'venas',
 'uefs',
 'celismara',
 'subaé',
 'cedraz',
 'vaccarezza',
 'ebda',
 'ix',
 'cifs',
 'acefs',
 'sincont',
 'sicomfs',
 'setcarfs',
 'xv',
 'délcio',
 'adalton',
 'sindihr',
 'xx',
 'sindvest',
 'ane',
 'meirevam',
 'claudiana',
 'edmeia',
 'ildes',
 'senaes',
 'comprodec',
 'obpi',
 'arcênio',
 'suzart',
 'osmário',
 'aidil',
 'giselle',
 'pregoeira',
 'merces',
 'elidiane',
 'sirleide',
 'dalei',
 'orgânicado',
 'capítuloi',
 'pmi',
 'ppps',
 'cgpm',
 'ueppp',
 'gts',
 'capítuloii',
 'capítuloiii',
 'combiná',
 'encaminhá',
 'los',
 'capítuloiv',
 'evantamentos',
 'capítulov',
 'tama',
 'lt',
 'issqn',
 'subempreitada',
 'spe',
 'reinvidicações',
 'dival',
 'mensag',
 'izaltina',
 'claudemiro',
 'pelúcio',
 'elisiana',
 'lesgisl',
 'anisía',
 'datilog',
 'financ',
 'uilma',
 'doranei',
 'zenilda',
 'eunira',
 'deb',
 'ofic',
 'egisl',
 'ol',
 'erideth',
 'tesour',
 'motorist',
 'smt',
 'apresentá',
 'identific

In [67]:
from spellchecker import SpellChecker
spellc = SpellChecker(language='pt')
erradas = spellc.unknown(['dalei', 'orgânicado', 'capítuloi', 'capítuloiii', 'combiná', 'encaminhá', 'los', 
                            'reinvidicações', 'mensag', 'bonito', 'orgânicamunicipal', 'segundafeira', 'jayana'])
# Não pode tirar hifem no meio de palavras. Consertar no método limpa texto
for palavra in erradas:
    print(f'Palavra da vez: {palavra}')
    print(spellc.correction(palavra))
    print(spellc.candidates(palavra))

Palavra da vez: dalei
falei
{'darei', 'daley', 'dali', 'dalai', 'dalek', 'falei', 'dale'}
Palavra da vez: segundafeira
segundafeira
{'segundafeira'}
Palavra da vez: orgânicado
organizado
{'orgânica', 'organizado', 'orgânico', 'orgânicas'}
Palavra da vez: jayana
sacana
{'mañana', 'aayan', 'bacana', 'mayans', 'katana', 'banana', 'jayne', 'catana', 'abana', 'jana', 'havana', 'joana', 'alana', 'juana', 'jaffna', 'savana', 'sacana', 'rayna', 'cabana', 'jaya', 'mayan'}
Palavra da vez: combiná
combina
{'combina', 'combine'}
Palavra da vez: encaminhá
encaminhar
{'encaminhar'}
Palavra da vez: orgânicamunicipal
orgânicamunicipal
{'orgânicamunicipal'}
Palavra da vez: reinvidicações
reivindicações
{'reivindicações'}
Palavra da vez: capítuloi
capítulo
{'capítulo', 'capítulos'}
Palavra da vez: capítuloiii
capítuloiii
{'capítuloiii'}
Palavra da vez: mensag
mensal
{'mensa', 'mensal'}
